# Finding the QAOA+cvar configuration

In [1]:
import os
import json

Here we upload the files data and calculate the min cost, max cost, min cost fev. number and max cost fev number. It is important to consider that:

* the cost cost_fev_number and nfev data of a QAOA hyperparameters corresponds to the mean value over the $5$ samples each configuration has.
* The min-max values mentioned will be used in an optimization functionto find the best QAOA hyperparameters.

In [2]:
def check_minmax(i,cost,cost_fev_number,file,minmax_data):
    '''Checks if value is the new minvalue o maxvalue. If that the case, replace the minmaxs.'''
    (min_cost,max_cost,
     min_cost_fev_number,max_cost_fev_number,
     min_cost_file,max_cost_file,
     min_cost_fev_number_file,max_cost_fev_number_file) =  minmax_data
    # check minmax cost
    if min_cost[i] is None or min_cost[i] > cost:
        min_cost[i] = cost
        min_cost_file[i] = file
    if max_cost[i] is None or max_cost[i] < cost:
        max_cost[i] = cost
        max_cost_file[i] = file
    # check minmax cost fev number  
    if min_cost_fev_number[i] is None or min_cost_fev_number[i] > cost_fev_number:
        min_cost_fev_number[i] = cost_fev_number
        min_cost_fev_number_file[i] = file
    if max_cost_fev_number[i] is None or max_cost_fev_number[i] < cost_fev_number:
        max_cost_fev_number[i] = cost_fev_number
        max_cost_fev_number_file[i] = file

def read_files_and_find_minmaxvalues(results,problems_directories,bechmarktype_director):
    # define de min-max structures
    min_cost = [None,None,None,None,None]
    max_cost =  [None,None,None,None,None]
    min_cost_fev_number =  [None,None,None,None,None]
    max_cost_fev_number =  [None,None,None,None,None]
    min_cost_file = [None,None,None,None,None]
    max_cost_file =  [None,None,None,None,None]
    min_cost_fev_number_file =  [None,None,None,None,None]
    max_cost_fev_number_file =  [None,None,None,None,None]
    minmax_data = (min_cost,max_cost
                   ,min_cost_fev_number,max_cost_fev_number
                   ,min_cost_file,max_cost_file
                   ,min_cost_fev_number_file,max_cost_fev_number_file)
    # iterate over the problem directories
    for problem_index,problem_dir in enumerate(problems_directories):
        
        # form the complete directory for the current problem
        complete_dir = 'results_cvar_qaoa/'+problem_dir+'/'+bechmarktype_director
    
        # initialize problem substructure
        results[problem_dir] = {}
    
        # iterate over each file in the problem directory
        for filename in os.listdir(complete_dir):
            print(problem_dir,'-',filename,end='\r')
            # guarante that the file is a json and is not evaluating the random initialization
            if filename.endswith('.json') and (not 'rand' in filename) and ('COBYLA' in filename):
                # create the complete filepath and the shortened filename (that is, just showing QAOA hyperparameters + cvar)
                file_path = os.path.join(complete_dir, filename)
                shortened_filename = filename[4:-5]
    
                # open the file
                with open(file_path, 'r', encoding='utf-8') as file:
                    try:
                        # load the file as a dict and get its relevant information
                        jsondata = json.load(file) 
                        cost = jsondata['optimized']['cost']
                        cost_fev_number = jsondata['optimized']['eval_number']
                        nfev = jsondata['evals']['number_of_evals']
    
                        # check if the current cost and cost fev. number are the new min or max.   
                        check_minmax(problem_index,cost,cost_fev_number,shortened_filename,minmax_data)
    
                        # update the cost, cost fev number and nfev sumatories, and the min-max values if neccesary
                        results[problem_dir][shortened_filename] = {}
                        results[problem_dir][shortened_filename]['cost'] = cost
                        results[problem_dir][shortened_filename]['cost_fev_number'] = cost_fev_number
                        results[problem_dir][shortened_filename]['nfev'] = nfev
                        del jsondata
                    except json.JSONDecodeError as e:
                        print(f"Error uploading {file_path}: {e}")
    
    print('\n-----------------')
    # iterate over each problem and QAOA hyperparameters (file data)
    for problem_index,(problemname,problem) in enumerate(results.items()):
        # print the min-max values of the problem
        print(problemname)
        print('min_cost:',min_cost[problem_index])
        print('max_cost:',max_cost[problem_index])
        print('min_cost_fev_number:',min_cost_fev_number[problem_index])
        print('max_cost_fev_number:',max_cost_fev_number[problem_index])
        print('min_cost_file:',min_cost_file[problem_index])
        print('max_cost_file:',max_cost_file[problem_index])
        print('min_cost_fev_number_file:',min_cost_fev_number_file[problem_index])
        print('max_cost_fev_number_file:',max_cost_fev_number_file[problem_index])
        print()
    print('-----------------')
    return  (results
            ,(min_cost
             ,max_cost
             ,min_cost_fev_number
             ,max_cost_fev_number
             ,min_cost_file
             ,max_cost_file
             ,min_cost_fev_number_file
             ,max_cost_fev_number_file
             ))

In [3]:
problems_directories = ['prob0','prob1','prob2','prob3','prob4'] # directorioes for each problem

problems_noiselesscvars = {} # data structure for all the noiselesscvars results
problems_noiselessexp = {} # data structure for all the noiselessexp results
problems_noisycvars = {} # data structure for all the noiselesscvars results
problems_noisyexp = {} # data structure for all the noiselessexp results


problems_noiselesscvars, minmax_data_noiselesscvars = read_files_and_find_minmaxvalues(problems_noiselesscvars,problems_directories,'noiselessCVAR')

problems_noiselessexp, minmax_data_noiselessexp = read_files_and_find_minmaxvalues(problems_noiselessexp,problems_directories,'noiselessEXP')

prob4 - prob0.5_p5-standard-rand-xy-COBYLA_noiseless.jsonjsonn
-----------------
prob0
min_cost: -7.6984
max_cost: -1.2142
min_cost_fev_number: 9
max_cost_fev_number: 200
min_cost_file: 0.5_p5-standard-ramp-x-COBYLA_noiseless
max_cost_file: 0.1_p4-extended-ramp-xy-COBYLA_noiseless
min_cost_fev_number_file: 0.2_p1-standard-ramp-xy-COBYLA_noiseless
max_cost_fev_number_file: 0.5_p3-extended-ramp-x-COBYLA_noiseless

prob1
min_cost: -13.6324
max_cost: -1.4216
min_cost_fev_number: 8
max_cost_fev_number: 200
min_cost_file: 0.3_p5-standard-ramp-x-COBYLA_noiseless
max_cost_file: 0.1_p3-standard-ramp-xy-COBYLA_noiseless
min_cost_fev_number_file: 0.4_p1-standard-ramp-xy-COBYLA_noiseless
max_cost_fev_number_file: 0.2_p1-extended-ramp-xy-COBYLA_noiseless

prob2
min_cost: -9.4248
max_cost: -1.37
min_cost_fev_number: 15
max_cost_fev_number: 200
min_cost_file: 0.3_p5-standard-ramp-x-COBYLA_noiseless
max_cost_file: 0.1_p4-standard-ramp-xy-COBYLA_noiseless
min_cost_fev_number_file: 0.2_p1-standard-ramp-

We run an optimization over function $f(cost,cost\_fev\_number) = \alpha \cdot minmax(cost) + \beta \cdot minmax(cost\_fev\_number)$. The $\alpha$ and $beta$ multipliers define how important is to consider each parameter. The minmax normalize over the discrete set of possible values for the parameters.

In [2]:
def min_max(x, min_val, max_val):
    return (x - min_val) / (max_val - min_val)

def f(cost,cost_fev_number,minmax_data,i):
    '''cost function for finding best QAOA configuration'''
    (min_cost,max_cost,
     min_cost_fev_number,max_cost_fev_number,
     _,_,_,_) =  minmax_data
    alpha=0.8 # weight of cost found by QAOA
    beta=0.2 # weights of cost fev number required by QAOA
    return alpha*min_max(cost,min_cost[i],max_cost[i]) + beta*min_max(cost_fev_number,min_cost_fev_number[i],max_cost_fev_number[i])

def find_minf(main_data,minmax_data):
    minf = [None,None,None,None,None]
    conf_minf = [None,None,None,None,None]
    cost_minf= [None,None,None,None,None]
    cost_fev_number_minf = [None,None,None,None,None]
    
    for problem_index,(problemname,problem) in enumerate(main_data.items()):
        for confname,conf in problem.items():
            fvalue = f(conf['cost'],conf['cost_fev_number'],minmax_data,problem_index)
            if minf[problem_index] is None or minf[problem_index] > fvalue:
                minf[problem_index] = fvalue
                conf_minf[problem_index] = confname
                cost_minf[problem_index] = conf['cost']
                cost_fev_number_minf[problem_index] = conf['cost_fev_number']
    
    for problem_index,(problemname,problem) in enumerate(main_data.items()):
        # print the min-max values of the problem
        print(problemname)
        print('min f value:',minf[problem_index])
        print('configuration of min f value:',conf_minf[problem_index])
        print('cost for min f value:',cost_minf[problem_index])
        print('cost fev number for min f value:',cost_fev_number_minf[problem_index])
        print()
    return (minf,conf_minf,cost_minf,cost_fev_number_minf)

In [5]:
print('NOISELESS_CVAR')
minf_data_noiselesscvars = find_minf(problems_noiselesscvars,minmax_data_noiselesscvars)
print('-------------------------')
print('NOISELESS_EXP')
minf_data_noiselessexp = find_minf(problems_noiselessexp,minmax_data_noiselessexp)

NOISELESS_CVAR
prob0
min f value: 0.09947643979057592
configuration of min f value: 0.5_p5-standard-ramp-x-COBYLA_noiseless
cost for min f value: -7.6984
cost fev number for min f value: 104

prob1
min f value: 0.08921740044332342
configuration of min f value: 0.5_p4-standard-ramp-x-COBYLA_noiseless
cost for min f value: -13.1928
cost fev number for min f value: 66

prob2
min f value: 0.09761146954174717
configuration of min f value: 0.4_p5-standard-ramp-x-COBYLA_noiseless
cost for min f value: -9.1604
cost fev number for min f value: 81

prob3
min f value: 0.10277676897966687
configuration of min f value: 0.2_p5-standard-ramp-x-COBYLA_noiseless
cost for min f value: -9.4672
cost fev number for min f value: 86

prob4
min f value: 0.0959480532092308
configuration of min f value: 0.3_p5-standard-ramp-x-COBYLA_noiseless
cost for min f value: -10.2648
cost fev number for min f value: 88

-------------------------
NOISELESS_EXP
prob0
min f value: 0.06344086021505377
configuration of min f v